In [ ]:
pip install --upgrade pip
pip install openai

In [ ]:
from openai import OpenAI
import os

In [ ]:
nome_tabela = "YourCatalog.YourSchema.YourTableName"
pergunta = "YourQuestion"

In [ ]:
def gerar_estrutura_tabela_string(tabela):
    schema = spark.sql("DESCRIBE " + tabela)
    schema = schema.filter(schema['col_name'].isNotNull())
    colunas_descricao = [
        f"{row['col_name']} ({row['data_type']}): {row['comment'] or 'Sem descrição'}"
        for row in schema.collect()
    ]
    estrutura = "\n".join(colunas_descricao)
    return estrutura

In [ ]:
def gerar_sql_com_claude(pergunta, estrutura, nome_tabela):
    prompt = f"""
Você é um assistente de dados. A tabela `{nome_tabela}` possui a seguinte estrutura:
{estrutura}
Com base nisso, gere uma query SQL Spark para responder à pergunta abaixo.
Pergunta: {pergunta}
Apenas retorne a query SQL. Não explique.
"""
    client = OpenAI(
    api_key='YourAPIkey',
    base_url="https://YourWorkspace.azuredatabricks.net/serving-endpoints"
    )
    response = client.chat.completions.create(
    model="databricks-claude-3-7-sonnet",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
    )
    return response.choices[0].message.content

In [ ]:
sql_gerada = gerar_sql_com_claude(pergunta, estrutura, tabela).replace('```sql\n','').replace('```','').strip()
print("SQL gerada:")
print(sql_gerada)
df = spark.sql(sql_gerada)
df.display()